In [2]:
from pathlib import Path
import re
import pandas as pd

In [11]:
df = pd.read_csv('gsm8k_with_gpt_responses.csv')

In [35]:
df["davinci_cot"].apply(lambda x: x.split(".")[-2])

0                  In total, she sold 48 + 24 = 72 clips
1                                       The answer is 10
2                                       The answer is 55
3                                       The answer is 48
4             So he writes 3 x 2 x 52 = 312 pages a year
                             ...                        
995                    Anna was 58 when they got married
996                                                    8
997      But after they have split the fish, they giv...
998                                            4 minutes
999                                    The answer is -40
Name: davinci_cot, Length: 1000, dtype: object

In [10]:
df["babbage_cot"].apply(lambda x: x.split(". ")[-1])[9]

'The answer is $36.00.'

In [18]:
[df[col].str.findall(r"The answer is.+$") for col in ["babbage_no_cot", "babbage_cot", "davinci_no_cot", "davinci_cot"]]

[0        [The answer is 24.]
 1         [The answer is 5.]
 2        [The answer is 20.]
 3        [The answer is 24.]
 4        [The answer is 52.]
                ...          
 995      [The answer is 65.]
 996    [The answer is  $63.]
 997      [The answer is 10.]
 998       [The answer is 5.]
 999      [The answer is $3.]
 Name: babbage_no_cot, Length: 1000, dtype: object,
 0              [The answer is 24.]
 1             [The answer is 550.]
 2      [The answer is 50 dollars.]
 3             [The answer is 108.]
 4                               []
                   ...             
 995            [The answer is 34.]
 996            [The answer is 70.]
 997                             []
 998             [The answer is 6.]
 999           [The answer is 100.]
 Name: babbage_cot, Length: 1000, dtype: object,
 0        [The answer is 72.]
 1        [The answer is $6.]
 2        [The answer is 25.]
 3        [The answer is 60.]
 4       [The answer is 624.]
                ...    

In [12]:
responses = pd.DataFrame(columns=["babbage_no_cot", "babbage_cot", "davinci_no_cot", "davinci_cot"])

In [13]:
responses["babbage_no_cot"] = df["babbage_no_cot"].str.findall(r"The answer is.+$")
responses["babbage_no_cot"] = responses["babbage_no_cot"].apply(lambda x: x[0] if x else "")
responses["babbage_cot"] = df["babbage_cot"].apply(lambda x: x.split(". ")[-1])
responses["davinci_no_cot"] = df["davinci_no_cot"].str.findall(r"The answer is.+$")
responses["davinci_no_cot"] = responses["davinci_no_cot"].apply(lambda x: x[0] if x else "")
responses["davinci_cot"] = df["davinci_cot"].apply(lambda x: x.split(". ")[-1])

In [14]:
responses

,babbage_no_cot,babbage_cot,davinci_no_cot,davinci_cot
0,The answer is 24.,The answer is 24.,The answer is 72.,"In total, she sold 48 + 24 = 72 clips."
1,The answer is 5.,The answer is 550.,The answer is $6.,The answer is 10.
2,The answer is 20.,The answer is 50 dollars.,The answer is 25.,The answer is 55.
3,The answer is 24.,The answer is 108.,The answer is 60.,The answer is 48.
4,The answer is 52.,"So James writes 10,000 pages a year.",The answer is 624.,So he writes 3 x 2 x 52 = 312 pages a year.
...,...,...,...,...
995,The answer is 65.,The answer is 34.,The answer is 22.,Anna was 58 when they got married.
996,The answer is $63.,The answer is 70.,The answer is $116.,The answer is 99.8.
997,The answer is 10.,How many fish will each of them be given to e...,The answer is 5.,How many fish will each of them be given to e...
998,The answer is 5.,The answer is 6.,The answer is 37.5.,The answer is 37.4 minutes.


In [27]:
responses.to_csv('gpt_gsmk8_outputs.csv', index=False)

In [57]:
df = pd.read_csv('gpt_gsmk8_outputs.csv')

In [15]:
df = responses

In [18]:
og_data = pd.read_csv('gsm8k.csv')

In [19]:
og_data = og_data[:len(df)]

In [20]:
df = og_data.join(df)

In [21]:
answers = df["answer"].str.findall(r"#### (\d+)$").apply(lambda x: x[0] if len(x) != 0 else None)

In [22]:
answers = answers.to_list()

In [30]:
def compute_accuracy(df, no_cot_col, cot_col, answer_col):
    no_cot_answers = df[no_cot_col].to_list()
    cot_answers = df[cot_col].to_list()
    answers = (
        df[answer_col]
        .str.findall(r"#### (\d+)$")
        .apply(lambda x: x[0] if len(x) != 0 else None)
    )
    answers = answers.to_list()
    answers = zip(no_cot_answers, cot_answers, answers)
    answers = list(answers)
    answers = [[str(a[0]), str(a[1]), str(a[2])] for a in answers]
    answers = [
        [
            re.findall(r"[^1-9](" + answer[2] + r")[^1-9]", answer[0]),
            re.findall(r"[^1-9](" + answer[2] + r")[^1-9]", answer[1]),
            answer[2],
        ]
        for answer in answers
    ]
    answers = [[len(answer[0]), len(answer[1]), answer[2]] for answer in answers]
    no_cot = 0
    cot = 0
    for a, b, c in answers:
        if a != 0:
            no_cot += 1
        if b != 0:
            cot += 1

    no_cot_percent = no_cot / len(answers) * 100
    cot_percent = cot / len(answers) * 100

    print(f"No COT accuracy: {no_cot_percent}%")
    print(f"COT accuracy: {cot_percent}%")

In [31]:
compute_accuracy(df, "babbage_no_cot", "babbage_cot", "answer")

No COT accuracy: 3.5000000000000004%
COT accuracy: 3.4000000000000004%


In [32]:
compute_accuracy(df, "davinci_no_cot", "davinci_cot", "answer")

No COT accuracy: 12.6%
COT accuracy: 36.199999999999996%
